In [5]:
from data_preparation import load_raw_data

data = load_raw_data()

# Extract the races DataFrame
races = data['races']

# Get unique combinations of 'name' and 'year'
unique_combinations = races[['name', 'year']].drop_duplicates()

print(unique_combinations)


/Users/I551659/Documents/GitHub/IE650-RAMP/ie500-data-mining-group7/lap_simulation/data_preparation.py:28: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  practice_sessions = pd.read_csv('../data/raw_data/ff1_laps.csv', na_values=na_values)
/Users/I551659/Documents/GitHub/IE650-RAMP/ie500-data-mining-group7/lap_simulation/data_preparation.py:29: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  tire_data = pd.read_csv('../data/raw_data/ff1_laps.csv', na_values=na_values)


                        name  year
0      Australian Grand Prix  2009
1       Malaysian Grand Prix  2009
2         Chinese Grand Prix  2009
3         Bahrain Grand Prix  2009
4         Spanish Grand Prix  2009
...                      ...   ...
1120  Mexico City Grand Prix  2024
1121    São Paulo Grand Prix  2024
1122    Las Vegas Grand Prix  2024
1123        Qatar Grand Prix  2024
1124    Abu Dhabi Grand Prix  2024

[1125 rows x 2 columns]


In [6]:
weather_data = data['weather_data']

# Display all distinct values
distinct_values = weather_data[['EventName', 'Year']].drop_duplicates()
print(distinct_values)

                       EventName  Year
0             Spanish Grand Prix  2024
472          Austrian Grand Prix  2024
906           British Grand Prix  2024
1382        Hungarian Grand Prix  2024
1885          Belgian Grand Prix  2024
2348            Dutch Grand Prix  2024
2819          Italian Grand Prix  2024
3272       Azerbaijan Grand Prix  2024
3758        Singapore Grand Prix  2024
4255    United States Grand Prix  2024
4710      Mexico City Grand Prix  2024
5221        São Paulo Grand Prix  2024
5749        Las Vegas Grand Prix  2024
6317            Qatar Grand Prix  2024
6457          Bahrain Grand Prix  2023
6968    Saudi Arabian Grand Prix  2023
7443       Australian Grand Prix  2023
7988       Azerbaijan Grand Prix  2023
8487            Miami Grand Prix  2023
8974           Monaco Grand Prix  2023
9462          Spanish Grand Prix  2023
9950         Canadian Grand Prix  2023
10448        Austrian Grand Prix  2023
10929         British Grand Prix  2023
11417       Hungarian Gra

In [7]:
import pandas as pd
import re
from data_preparation import load_raw_data
from thefuzz import process  # Optional for fuzzy matching

# Load data
data = load_raw_data()
races = data['races']
weather_data = data['weather_data']  # Adjust the key if different

# Preprocessing function
def preprocess_string(s):
    if isinstance(s, str):
        s = s.lower().strip()
        s = re.sub(r'\s+', ' ', s)
        s = re.sub(r'grand prix', 'gp', s)
        s = re.sub(r'[^\w\s]', '', s)
        return s
    return s

# Apply preprocessing
weather_data['EventName_clean'] = weather_data['EventName'].apply(preprocess_string)
races['name_clean'] = races['name'].apply(preprocess_string)

# Merge to find matches
merged = weather_data.merge(
    races,
    left_on=['EventName_clean', 'Year'],
    right_on=['name_clean', 'year'],
    how='left',
    indicator=True
)

# Summary
total_weather_races = weather_data.shape[0]
matched_races = merged['_merge'].value_counts().get('both', 0)
unmatched_races = merged['_merge'].value_counts().get('left_only', 0)

print(f"Total races in weather data: {total_weather_races}")
print(f"Matched races: {matched_races}")
print(f"Unmatched races: {unmatched_races}")

# Handle unmatched races
if unmatched_races > 0:
    print("\nUnmatched Races:")
    print(merged[merged['_merge'] == 'left_only'][['EventName', 'Year']].drop_duplicates())

    # Example: Fuzzy matching (optional)
    unmatched_event_names = merged[merged['_merge'] == 'left_only']['EventName_clean'].unique()
    races_event_names = races['name_clean'].unique()

    for event in unmatched_event_names:
        match, score = process.extractOne(event, races_event_names)
        print(f"Weather Event: {event} --> Races Match: {match} (Score: {score})")
else:
    print("\nAll races in weather data are present in races data.")

# Final join (if all matched or after resolving mismatches)
final_merged = merged[merged['_merge'] == 'both']
print("\nFinal Merged DataFrame:")
print(final_merged.head())


/Users/I551659/Documents/GitHub/IE650-RAMP/ie500-data-mining-group7/lap_simulation/data_preparation.py:28: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  practice_sessions = pd.read_csv('../data/raw_data/ff1_laps.csv', na_values=na_values)
/Users/I551659/Documents/GitHub/IE650-RAMP/ie500-data-mining-group7/lap_simulation/data_preparation.py:29: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  tire_data = pd.read_csv('../data/raw_data/ff1_laps.csv', na_values=na_values)


Total races in weather data: 28154
Matched races: 28154
Unmatched races: 0

All races in weather data are present in races data.

Final Merged DataFrame:
                     Time  AirTemp  Humidity  Pressure  Rainfall  TrackTemp  \
0  0 days 00:00:25.642000     27.5      37.0    1004.3     False       46.3   
1  0 days 00:01:25.662000     27.5      36.0    1004.5     False       46.4   
2  0 days 00:02:25.653000     27.5      36.0    1004.5     False       47.3   
3  0 days 00:03:25.660000     27.6      34.0    1004.4     False       47.5   
4  0 days 00:04:25.667000     27.7      35.0    1004.4     False       46.4   

   WindDirection  WindSpeed  Year           EventName  ...    fp2_date  \
0             93        0.0  2024  Spanish Grand Prix  ...  2024-06-21   
1              0        0.0  2024  Spanish Grand Prix  ...  2024-06-21   
2            241        2.3  2024  Spanish Grand Prix  ...  2024-06-21   
3            226        2.0  2024  Spanish Grand Prix  ...  2024-06-21   
4